In [1]:
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

In [2]:
class FeatureTargetTransformer:
    def __init__(self, feature_columns=['High', 'Low', 'Open', '20_EMA', '50_EMA', '200_EMA'], target_column='Close'):
        self.feature_columns = feature_columns
        self.target_column = target_column

    def transform(self, data):
        # Extract features and target
        X = data[self.feature_columns].values
        y = data[self.target_column].values

        # Scale the data
        scaler = MinMaxScaler(feature_range=(0, 1))
        X_scaled = scaler.fit_transform(X)
        y_scaled = scaler.fit_transform(y.reshape(-1, 1))

        # Reshape data for LSTM (samples, time steps, features)
        X_reshaped = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

        return X_reshaped, y_scaled, scaler

# Load data from CSV file
data = pd.read_csv('data_one_last_time.csv')

# Instantiate transformer
transformer = FeatureTargetTransformer()

# Transform data
X, y, scaler = transformer.transform(data)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Make predictions on the test set
y_pred_scaled = model.predict(X_test)

# Inverse scale the predictions
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_orig = scaler.inverse_transform(y_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test_orig, y_pred)
print("Mean Squared Error:", mse)

Epoch 1/100
133/133 [==============================] - 6s 5ms/step - loss: 0.0263
Epoch 2/100
133/133 [==============================] - 1s 4ms/step - loss: 4.6672e-04
Epoch 3/100
133/133 [==============================] - 1s 4ms/step - loss: 2.3787e-04
Epoch 4/100
133/133 [==============================] - 1s 4ms/step - loss: 1.5217e-04
Epoch 5/100
133/133 [==============================] - 1s 5ms/step - loss: 1.2599e-04
Epoch 6/100
133/133 [==============================] - 1s 4ms/step - loss: 1.0550e-04
Epoch 7/100
133/133 [==============================] - 1s 5ms/step - loss: 8.7724e-05
Epoch 8/100
133/133 [==============================] - 1s 4ms/step - loss: 6.9415e-05
Epoch 9/100
133/133 [==============================] - 1s 4ms/step - loss: 5.2646e-05
Epoch 10/100
133/133 [==============================] - 1s 4ms/step - loss: 3.8623e-05
Epoch 11/100
133/133 [==============================] - 1s 4ms/step - loss: 2.8850e-05
Epoch 12/100
133/133 [==============================] - 

In [3]:
r2 = r2_score(y_test_orig, y_pred)

print("R^2 Score:", r2)

R^2 Score: 0.9998495772861734
